# Build a Question Answer Application using Graph DB neo4j

In [5]:
import os
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph

In [4]:

os.environ["NEO4J_URI"] = os.getenv('NEO4J_URI')
os.environ["NEO4J_USERNAME"] = os.getenv('NEO4J_USERNAME')
os.environ["NEO4J_PASSWORD"] = os.getenv('NEO4J_PASSWORD')

'neo4j+s://0939c4f3.databases.neo4j.io'

In [6]:
url = 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/refs/heads/main/movies/movies.csv'
db = Neo4jGraph(url = os.environ["NEO4J_URI"],
                 username=os.environ["NEO4J_USERNAME"],
                 password=os.environ["NEO4J_PASSWORD"],
                 refresh_schema=True )

In [7]:
db

In [23]:
query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""
result = db.query(query)
result

[]

In [35]:
db.refresh_schema

<bound method Neo4jGraph.refresh_schema of <langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7fd8d1ccbb20>>

In [36]:
print(db.schema)

Node properties:
Territory {territoryID: STRING, territoryDescription: STRING}
Region {regionID: STRING, regionDescription: STRING}
Supplier {supplierID: STRING, companyName: STRING, contactName: STRING, contactTitle: STRING, address: STRING, city: STRING, region: STRING, postalCode: STRING, country: STRING, phone: STRING, fax: STRING, homePage: STRING}
Shipper {companyName: STRING, phone: STRING, shipperID: STRING}
Product {productID: STRING, productName: STRING, unitPrice: FLOAT, unitsInStock: INTEGER, unitsOnOrder: INTEGER, reorderLevel: INTEGER, discontinued: BOOLEAN}
Category {categoryID: STRING, categoryName: STRING, description: STRING}
Order {orderID: STRING, orderDate: STRING, requiredDate: STRING, shippedDate: STRING, freight: STRING, shipName: STRING, shipAddress: STRING, shipCity: STRING, shipRegion: STRING, shipPostalCode: STRING, shipCountry: STRING}
Customer {companyName: STRING, contactName: STRING, contactTitle: STRING, address: STRING, city: STRING, region: STRING, po

In [40]:
groq_api_key = os.getenv('GROQ_API_KEY')
from langchain_groq import ChatGroq
llm = ChatGroq(model="Gemma2-9b-it",groq_api_key= groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fd8be414a60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fd8be417040>, model_name='Gemma2-9b-it')

In [41]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(graph= db,llm=llm,verbose=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7fd8d1ccbb20>, cypher_generation_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'schema'], template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fd8be414a60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fd8be417040>, model_name='Gemma2-9b-it')), qa_chain=LLMChain(prompt=PromptTemplate(input_v

In [42]:
response = chain.invoke({"query":"Who was the director of the movie Casino"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie {title: "Casino"})-[:DIRECTED_BY]->(d:Director)
RETURN d.name



Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: DIRECTED_BY)} {position: line: 2, column: 37, offset: 43} for query: 'cypher\nMATCH (m:Movie {title: "Casino"})-[:DIRECTED_BY]->(d:Director)\nRETURN d.name\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your app

Full Context:
[]

> Finished chain.


In [43]:
response=chain.invoke({"query":"Who were the actors of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Actor)<-[:ACTED_IN]-(m:Movie {title: "Casino"}) RETURN a.name 



Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: Actor)} {position: line: 1, column: 10, offset: 9} for query: 'MATCH (a:Actor)<-[:ACTED_IN]-(m:Movie {title: "Casino"}) RETURN a.name \n'


Full Context:
[]

> Finished chain.


{'query': 'Who were the actors of the movie Casino',
 'result': "I don't know the answer. \n"}

In [44]:
response=chain.invoke({"query":"How many artists are there?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Artist)
RETURN count(a) 





Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: Artist)} {position: line: 1, column: 10, offset: 9} for query: 'MATCH (a:Artist)\nRETURN count(a) \n\n\n'


Full Context:
[{'count(a)': 0}]

> Finished chain.


{'query': 'How many artists are there?', 'result': 'There are 0 artists. \n'}

In [45]:
response=chain.invoke({"query":"How many movies has Tom Hanks acted in"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Movie)<-[:ACTED_IN]-(t:Person {name:"Tom Hanks"}) RETURN count(n) 


Full Context:
[{'count(n)': 2}]

> Finished chain.


{'query': 'How many movies has Tom Hanks acted in',
 'result': "I don't know the answer. \n"}